<a href="https://colab.research.google.com/github/Aidan1223f/SERP_2022/blob/main/3rd%20iteration%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

from google.colab import files
uploaded = files.upload()

Saving www-gutenberg-org-files-1342-1342-0-txt.txt to www-gutenberg-org-files-1342-1342-0-txt.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

Preproccessing::

In [ ]:
file = open("www-gutenberg-org-files-1342-1342-0-txt.txt", "r", encoding = "utf8")


line = []
for i in file:
    line.append(i)


txtData = ""
for i in line:
  txtData = ' '. join(line) 

txtData = txtData.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

 
txtData = txtData.split()
txtData = ' '.join(txtData)
print(txtData[:300])

print(len(txtData))

The Project Gutenberg eBook of Pride and prejudice, by Jane Austen This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gute
104345


Tokenizer:

In [ ]:
token = Tokenizer(oov_token='<OOV>')
token.fit_on_texts([txtData])

# saving the tokenizer for predict function
pickle.dump(token, open('token.pkl', 'wb'))

seq_data = token.texts_to_sequences([txtData])[0]
print(seq_data[:30])
print(len(seq_data))

[2, 646, 501, 375, 3, 138, 5, 307, 27, 67, 116, 39, 375, 14, 22, 2, 427, 3, 871, 1347, 7, 2, 502, 872, 5, 86, 99, 873, 3, 2]
18732


In [ ]:
vocabSize =  len(token.word_index) + 1
print(vocabSize)

3043


Creates sequence of 3 words and predicted word 

In [ ]:
seq = []
for i in range(4, len(seq_data)): 
  wrd = seq_data[i-4 : i + 1]
  seq.append(wrd)

print("sequence length: " , len(seq))
seq = np.array(seq)
print(seq[:15])

sequence length:  18728
[[  2 646 501 375   3]
 [646 501 375   3 138]
 [501 375   3 138   5]
 [375   3 138   5 307]
 [  3 138   5 307  27]
 [138   5 307  27  67]
 [  5 307  27  67 116]
 [307  27  67 116  39]
 [ 27  67 116  39 375]
 [ 67 116  39 375  14]
 [116  39 375  14  22]
 [ 39 375  14  22   2]
 [375  14  22   2 427]
 [ 14  22   2 427   3]
 [ 22   2 427   3 871]]


Data represents the first 3 letters that the alg will be reading 
:  Predict represents predicted word

In [ ]:
data = []
predict = []

for i in seq:
    data.append(i[0:4])
    predict.append(i[4])
    
data = np.array(data)
predict = np.array(predict)

print(data)
print("\n")
print(predict)


[[   2  646  501  375]
 [ 646  501  375    3]
 [ 501  375    3  138]
 ...
 [3042   45  178  234]
 [  45  178  234   84]
 [ 178  234   84   79]]


[   3  138    5 ...   84   79 1089]




converts the class vectors into binary class matrix




In [ ]:
predict = to_categorical(predict, num_classes=vocabSize)
print(predict[:5])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Model Creation: LSTM 

In [ ]:
model = Sequential()
model.add(Embedding(vocabSize, 10, input_length=4))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocabSize, activation="softmax"))

In [ ]:
print(model.summary())

In [ ]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(data, predict, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
293/293 [==============================] - ETA: 0s - loss: 6.6082
Epoch 1: loss improved from inf to 6.60818, saving model to next_words.h5
293/293 [==============================] - 202s 676ms/step - loss: 6.6082
Epoch 2/70
293/293 [==============================] - ETA: 0s - loss: 6.2311
Epoch 2: loss improved from 6.60818 to 6.23106, saving model to next_words.h5
293/293 [==============================] - 199s 679ms/step - loss: 6.2311
Epoch 3/70
293/293 [==============================] - ETA: 0s - loss: 5.9710
Epoch 3: loss improved from 6.23106 to 5.97097, saving model to next_words.h5
293/293 [==============================] - 200s 682ms/step - loss: 5.9710
Epoch 4/70
293/293 [==============================] - ETA: 0s - loss: 5.6742
Epoch 4: loss improved from 5.97097 to 5.67416, saving model to next_words.h5
293/293 [==============================] - 201s 684ms/step - loss: 5.6742
Epoch 5/70
293/293 [==============================] - ETA: 0s - loss: 5.3749
Epoch 5: lo

In [ ]:
# credits:  IG Tech Team
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-4:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: where are you 
['where', 'are', 'you', '']
1/1 [==============================] - 2s 2s/step
yes
Enter your line: what do you 
['what', 'do', 'you', '']
1/1 [==============================] - 0s 68ms/step
consider
Enter your line: I am going to 
['am', 'going', 'to', '']
1/1 [==============================] - 0s 52ms/step
i
Enter your line: where did you 
['where', 'did', 'you', '']
1/1 [==============================] - 0s 44ms/step
yes
Enter your line: what did you 
['what', 'did', 'you', '']
1/1 [==============================] - 0s 44ms/step
consider
Enter your line: I can not 
['I', 'can', 'not', '']
1/1 [==============================] - 0s 52ms/step
with
Enter your line: I will not 
['I', 'will', 'not', '']
1/1 [==============================] - 0s 58ms/step
that
Enter your line: I feel that 
['I', 'feel', 'that', '']
1/1 [==============================] - 0s 60ms/step
all
Enter your line: how does that 
['how', 'does', 'that', '']
1/1 [=========================